! jt -r

In [1]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime
import src.info as info
import src.in_out as io

from src.density import Density

1. **data** klasöründeki tüm dosya yolları alınıyor

In [2]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

2. *Her bir path'de bulunan veriler **dataframe** olarak alınıyor. Her bir df, **split_df** metodu ile saatlik olarak **23** ayrı df'e bölünüyor. Bölünmüş her bir df'in **mid_price**'ı hesaplanıp yeni bir sütun olarak **df**'e ekleniyor.* **get_info** metodu her bir path için bu verileri kullanarak bir **info** örneği oluşturup döndürür. Her bir **info** örneği, aşağıdaki özelliklere sahiptir:<br>
    * Okunan dosyanın tarihi
    * Ürünün adı
    * Okunan dosyaya ait yol
    * Tüm verileri içeren dataframe
    * Saatlik olarak bölünmüş dataframe listesi


In [3]:
infolist = [info.get_info(path) for path in path_list]

3. Burada, oluşturulan infolar arasında bir **eşleştirme** yapılmaktadır. Çünkü her bir **spread** hesabının hangi veriler kullanılarak yapılacağı bilinmelidir. Eşleşen infolar, tuple çiftleri olarak tutulmakta.
  Elimizde 2018:09:06 tarihinin 6A,6B,6C ürünlerine ait verileri olsun. Bu durumda 6A-6B, 6A-6C, 6B-6C arasında 3 adet spread hesaplanacaktır. Hesaplama öncesi bu eşleştirme, **daily_data** modülünün **to_match_days** metoduyla yapılmaktadır.

In [4]:
paired_days = dt.to_match_days(infolist)

4.  **daily_datas**: GünlükVeri örneklerinin tutulduğu liste <br />
**dt.to_match_hour** metodu ile her bir eşleşmiş çiftin mid_price'ları saatlik olarak alınıp birleştiriliyor. <br />
**gunluk_veri**: **tarih**, her bir saate ait **change**,**spread**,**amplitude**,**duration**,**urun adları**...vs içeriyor <br />
**paired_hours:** 23 adet df içerir. her bir df mid_price çifti içerir

In [5]:
daily_data = []

In [6]:
for pair in paired_days:
    product = pair[0].product + "_" + pair[1].product   # urun adi
    paired_hours=dt.to_match_hour(pair[0],pair[1])
    gunluk_veri = dt.DailyData(mid_price_list=paired_hours,date=pair[0].date, name= product)
    gunluk_veri.spread_hesapla()
    daily_data.append(gunluk_veri)

5. Her bir **DailyData** örneği ürünlere göre ayrı ayrı gruplandırılıyor. <br>
    (['**6AU8_6BU8**': [gunluk veriler...], '**6AU8_6CU8**':[gunluk veriler...], '**6BU8_6CU8**':[gunluk veriler...]])

In [7]:
products={}

In [8]:
for gunluk in daily_data:
    if gunluk.name in products:
        products[gunluk.name].append(gunluk)
    else:
        products[gunluk.name] = [gunluk]

**combined_days :**  key-value olarak ürün adları ve verileri içerir <br />
**combined_days_detail :** key-value olarak ürün adları ve verilere ait istatistiksel bilgileri içerir <br />
**combined_days_con :** key-value olarak ürün adları ve median koşuluna göre yeniden düzenlenmiş verileri içerir <br />
* 6BU8_6CU8'e ait verilere erişmek için:<br />
 ```python
combined_days_detail['6BU8_6CU8']
```
* 6BU8_6CU8'e ait 18:00:00-19:00:00 saatleri arası verilere erişmek için: <br />
 ```python
combined_days['6BU8_6CU8'].loc[time(18)]
```

In [9]:
combined_days = {}
combined_days_detail = {}
combined_days_con = {}
combined_days_con_detail = {}

In [10]:
stop=False
i=0
while products:
    urun = products.popitem()
    df = dt.get_all_data(urun[1]).dropna()
    con_df = con.scan(df)
    combined_days[urun[0]] = df
    combined_days_con[urun[0]] = con_df
#    combined_days_detail[urun[0]] = aux.get_detail(df)
#     combined_days_con_detail[urun[0]] = aux.get_detail(con_df)

In [11]:
combined_days_con['6BU8_6CU8']

duration  amplitude
18:00:00 0         1.0 -50000.000
         1         4.0     16.875
         4         8.0    -10.000
         5         2.0     10.000
         6         2.0    -10.000
         9        55.0     15.000
         10        3.0    -10.000
         19      188.0      8.125
         20      210.0    -28.125
         21       14.0      5.000
         22       25.0    -10.000
         23       46.0     10.000
         24      136.0     -8.125
         27      120.0      8.125
         28      121.0    -10.000
         29       69.0      5.000
         30      113.0    -15.000
         35      221.0      5.000
         36      110.0    -10.000
         39      140.0     10.000
         44       65.0     -8.125
         47       18.0      8.125
         54       36.0     -8.125
         59       83.0      3.125
         60        1.0    -10.000
         61        9.0     10.000
         62        3.0    -10.000
         69       21.0     10.000
         70        2.0     -8.125
         71        2.0      8.125
...                ...        ...
16:00:00 352      26.0     20.000
         353     148.0    -20.000
         354      15.0     10.000
         357      50.0    -20.000
         360     181.0     10.000
         361       8.0    -11.250
         362       7.0     10.000
         363      23.0    -15.000
         364       6.0     10.000
         365       3.0    -10.000
         366      19.0     11.250
         367      45.0    -11.250
         368      14.0      5.000
         369      15.0     -5.000
         370      14.0      6.250
         373      68.0     -6.250
         374      16.0     22.500
         375       4.0    -10.000
         376       2.0     20.000
         377       3.0    -16.250
         378       1.0     10.000
         381      44.0    -20.000
         382       4.0     53.125
         383       3.0    -25.000
         384       3.0     15.000
         385       1.0    -10.000
         386      12.0     25.000
         389       7.0    -20.000
         392      14.0     13.125
         395      10.0    -13.125

[9811 rows x 2 columns]

Hesaplanmış ürün adları ve saat bilgisi alınıyor

In [12]:
all_products = list(combined_days.keys())

In [13]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

all_density:
```json
{"6BU8_6CU8": {"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 "6AU8_6CU8":{"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 ....}
```

In [14]:
all_density= {}

In [15]:
for product in all_products:
    densities = {}
    for hour in hour_series:
        df = combined_days[product].loc[hour]
        densities[hour]= Density(df)
    all_density[product] = densities

--------------------------------

# Example

In [16]:
data_of_the_eighteenth_clock.density_data

NameError: name 'data_of_the_eighteenth_clock' is not defined

In [ ]:
data_of_the_eighteenth_clock = all_density['6AU8_6BU8'][time(18)]

data_of_the_eighteenth_clock.data

data_of_the_eighteenth_clock.density_data

In [ ]:
data_of_the_eighteenth_clock.draw()